# Qeuery_cell_column_index with duplacte observations

In [2]:
import pickle
import pandas as pd
import numpy as np
columns = ['EID', 'EDateTime', 'SubListID', 'ECategory', 'OrigName', 'Address', 'DOB', 'Nationality', 'Designation', 'Organization', 'Title', 'DO_Inclusion', 'DO_Exclusion', 'Aliases', 
           'Matching', 'ListType', 'Remarks', 'Status', 'Gender', 'Last_Occupation', 'Documents', 'Initials', 'Name', 'Image']
           
with open("../../../list_of_lists_scraped.pkl", "rb") as file:
    lst = pickle.load(file)    

for i in range(len(lst)):
    lst[i] = list(lst[i]) # the <lst> is list of tuples, and tuple is immutable, so we have to change it to list to modify it
    for j in range(len(lst[i])):
        if isinstance(lst[i][j], str):
            lst[i][j] = lst[i][j].replace("\n", "||n").\
                                    replace("\r", "||r").\
                                    replace("\t", "||t").\
                                    replace("\\u", "||u").\
                                    replace("\\x", "||x").\
                                    replace(";", ", ").strip().lower()


df = pd.DataFrame(lst, columns = columns)

for i in df:
    if df[i].dtype == "object":
        df[i] = list(df[i].str.lower())

In [3]:
words_list = ['ketamine', 'drug dealing', 'alcohol', 'substance abuse', 'drug trafficking', 'traficar drogas', 
              'el sporífero', '4-methyl methcathinone', 'poppy', 'kat', 'solvents', 'narco-traffickers', 'pregabalin', 
              'opium', 'narco trafficking', 'psilocybin', 'anabolic steroids', 'intoxicated', 'narcóticos', 'opiate', 
              'oxycodone', 'ecstasy', 'heroin', 'drogas', 'narca', 'drugs', 'nicotine', 'crystal meth', 'ketamine', 'drug', 
              'drug dealing', 'cocaína', 'narco', 'narcotics', 'methylamphetamine', 'buprenorphine', 'alcohol', 
              'dealing drugs', 'benzodiazepines', 'tobacco', 'benzodiazepines', 'el tráfico de drogas', 
              '4-methyl methcathinone', 'narco', 'ghb', 'lsd', 'narco-traffickers', 'white magic', 'm-cat', 'pcp', 
              'metanfetamina', 'mdma', 'amphetamines', 'psilocyn', 'narco trafficking', '4-mmc', 'drug abuse', 
              'narcotics traffickers', 'ghb', 'cannabis', 'magic mushrooms', 'cocaine', 'pwid', 'drug dealing', 'heroin', 
              'drug trafficking', 'khat', 'morphine', 'mephedrone', 'meow meow', 'drone', 'phencyclidine', 'bath salts', 
              'buprenorphine']

In [4]:
df.to_csv("master-table.csv")

In [5]:
columns_contains_query = []
index_numbers = []
QUERY = []
for query in words_list:
    query_only_word = f" {query} " # 1508 results
    for i in df.dtypes[df.dtypes == 'O'].index:
            if any(df[i].str.contains(query_only_word).unique()):
                m = np.where(df[i].str.contains(query_only_word))
                index_numbers += m[0].tolist()
                QUERY += [query] * len(m[0].tolist())
                columns_contains_query += ([i] * len(m[0].tolist()))               

adf = pd.DataFrame({"index_numbers" : index_numbers,
                   "QUERY" : QUERY,
                   "column_name" : columns_contains_query})
adf.head()

,index_numbers,QUERY,column_name
0,165717,alcohol,OrigName
1,16232,alcohol,Remarks
2,107257,alcohol,Remarks
3,108968,alcohol,Remarks
4,108980,alcohol,Remarks


In [6]:
adf.QUERY.value_counts()

drug                 426
drug trafficking     416
drugs                390
narcotics            263
tobacco              158
heroin                32
alcohol               18
cocaine               14
drogas                 9
solvents               8
kat                    5
cannabis               3
pcp                    2
narco-traffickers      2
morphine               1
oxycodone              1
opium                  1
intoxicated            1
poppy                  1
Name: QUERY, dtype: int64

In [7]:
cell = []
column_name = []
query = []
index_num = []

for i in adf.iterrows():
    ind,q,c_name = i[1]
    cell.append(df.loc[ind, c_name])
    column_name.append(c_name)
    query.append(q)
    index_num.append(ind)
    
bdf = pd.DataFrame({"Index" : index_num,
                    "Query" : query,
                    "Cell"  : cell,
                    "column": column_name})

bdf.head()

,Index,Query,Cell,column
0,165717,alcohol,star industrial alcohol (pvt) ltd.,OrigName
1,16232,alcohol,in addition to serving as uday saddam hussein’...,Remarks
2,107257,alcohol,post/zip code: other information: served as u...,Remarks
3,108968,alcohol,post/zip code: other information: served as u...,Remarks
4,108980,alcohol,post/zip code: other information: served as u...,Remarks


In [10]:
bdf.to_csv("Qeuery_cell_column_index.csv")

In [1]:
!wc -l Qeuery_cell_column_index.csv

1752 Qeuery_cell_column_index.csv


# Qeuery_cell_column_index without duplacte observations

In [8]:
grouped_unique_Query = bdf.groupby('Index')['Query'].apply(lambda x: "{%s}" % ', '.join(x))
grouped_unique_column = bdf.groupby('Index')['column'].apply(lambda x: "{%s}" % ', '.join(x))

a = grouped_unique_Query.str.replace("}", "").str.replace("{", "").str.strip().str.split(",")
b = grouped_unique_column.str.replace("}", "").str.replace("{", "").str.strip().str.split(",")
lst3 = []
for ii,BB in zip([a,b], [grouped_unique_Query, grouped_unique_column]):
    lst2 = []
    for i in ii:
        am = set(list(map(str.strip, i)))
        if len(am) > 1:
            lst2.append(am)
        else:
            lst2.append(list(am)[0])

    lst3.append([str(i) if (type(i) == set) else i for i in lst2])

cdf = pd.DataFrame({"Index" : grouped_unique_Query.index,
              "column" : lst3[1],
              "Query" :  lst3[0]})


cdf.head()

,Index,column,Query
0,2181,Remarks,"{'drug trafficking', 'drug'}"
1,2182,Remarks,"{'drug trafficking', 'drug'}"
2,2302,Address,kat
3,2510,Remarks,"{'drug trafficking', 'drug'}"
4,2543,Remarks,"{'drug trafficking', 'drug'}"


In [9]:
cdf.to_csv("Query_column_index-without-duplacte-observations.csv")

# Removing observations that contain one of [pvt, ltd, limited, private] after some characters after our Query word`

In [12]:
indexx = []
cols = []
for i in bdf.iterrows():
    ind, q, cel, column = i[1]
    if ("pvt" in cel[cel.index(q)+len(q):cel.index(q)+len(q)+8].lower()) \
    or ("ltd" in cel[cel.index(q)+len(q):cel.index(q)+len(q)+8].lower()) \
    or ("limited" in cel[cel.index(q)+len(q):cel.index(q)+len(q)+13].lower()) \
    or ("private" in cel[cel.index(q)+len(q):cel.index(q)+len(q)+13].lower()):
        indexx.append(ind)
        cols.append(column)
print("Observations shoul remove after this operation", len(indexx))
print(set(cols))

len(bdf)

Observations shoul remove after this operation 236
{'Organization', 'OrigName', 'Name'}


1751

In [13]:
ddf = bdf[~bdf.Index.isin(indexx)]
ddf.to_csv("after_removing_pvt_etc.csv")

# PCP only 2 times

In [14]:
ii = []
for i in bdf.iterrows():
    ind, q, cel, column = i[1]
    if "pcp" in cel:
        print(cel)

{"info": [{"aka": "shining path"}, {"aka": " partido comunista del peru en el sendero luminoso de jose carlos mariategui"}, {"aka": " communist party of peru on the shining path of jose carlos mariategui"}, {"aka": " partido comunista del peru"}, {"aka": " communist party of peru"}, {"aka": " the communist party of peru by the shining path of jose carlos mariategui and marxism"}, {"aka": " leninism"}, {"aka": " maoism and the thoughts of chairman gonzalo"}, {"aka": " revolutionary student front for the shining path of mariategui"}, {"aka": " communist party of peru - by way of the shining path of mariategui"}, {"aka": " pcp - por el sendero luminoso de mariategui"}, {"aka": " pcp and pcp-sl"}]}
{"info": [{"aka": "sp"}, {"aka": " sendero luminoso"}, {"aka": " the peruvian communist party"}, {"aka": " pcp-sl"}, {"aka": " pcp"}, {"aka": " sl"}, {"aka": " partido comunista del peru en el sendero luminoso de jose carlos mariategui"}, {"aka": " communist party of peru on the shining path of 

In [15]:
ddf.Index.nunique()

905